<center> <h1> Prompt Engineering with Open-Source Large Language Models (LLMs) using HuggingFace Serverless APIs</h1> </center>

<p style="margin-bottom:1cm;"></p>

_____


In this notebook we will learn how to run any open-source LLMs via HugginFace Inference APIs using this colab notebook. You can run this notebook in your local server also without worrying about having enough infrastructure to run these models!

Thankfully HuggingFace has made its [__Inference API__](https://huggingface.co/docs/api-inference/quicktour) free to use with some basic rate limits etc. in place so you don't end up making unlimited requests on it's servers.

The best part is you can access 150,000+ deep learning models without worrying about your infrastructure.

The models we will be trying here include:

- __[Mistral-7B-Instruct-v0.2](https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2)__ model which is a 7B parameters transformer LLM built by the French young company [MistralAI](https://mistral.ai/company/)  is a instruct fine-tuned version of the [Mistral-7B-Instruct-v0.1](https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1) which is based on their first [Mistral-7B-v0.1](https://huggingface.co/mistralai/Mistral-7B-v0.1) generative text model using a variety of publicly available conversation datasets.

- __[gemma-2b-it ](https://huggingface.co/google/gemma-1.1-2b-it)__ is a part of Google's gemma series, a 2 billion parameter transformer model fine-tuned for instruction-following tasks, enabling it to handle a wide array of complex language processing activities.



__You just need an internet connection and a HuggingFace Account and API Key to use these models.__


## Get your API Key

Remember to go to your [HuggingFace Account Settings](https://huggingface.co/settings/account) and generate an API key by creating a new token from the [Access Tokens](https://huggingface.co/settings/tokens) section.


## Load HuggingFace API Credentials

Enter your key from [here](https://huggingface.co/settings/tokens)

In [1]:
from getpass import getpass

API_KEY = getpass("Enter HuggingFace API Key: ")

Enter HuggingFace API Key:  ········


### Create LLM API Access Function

Here we create a basic function which can access any LLM API endpoint available on HuggingFace.

For more details refer to the [detailed documentation](https://huggingface.co/docs/api-inference/detailed_parameters#text-generation-task) as needed.

In [2]:
import requests

headers = {"Authorization": "Bearer "+API_KEY}

def query(payload, MODEL_API_URL):
  response = requests.post(MODEL_API_URL, headers=headers, json=payload)
  print('API Response:', response)
  return response.json()

## Create LLM API Access Config

Here we decide which LLMs we will access by getting their inference API endpoints.

We also set some general configuration settings. You can find the [detailed documentation](https://huggingface.co/docs/api-inference/detailed_parameters#text-generation-task) here.

Some useful config settings include:

- max_new_tokens: The amount of new tokens to be generated in the response
- do_sample: Whether or not to use sampling. False means use greedy decoding i.e temperature=0
- temperature: Between 0 - 1, The value used to module the next token probabilities. Higher temperature means the results may vary and be more creative
- return_full_text: If set to False, does not return your input prompt to the model
- wait_for_model:  If the model is not ready, wait for it instead of receiving 503. It limits the number of requests required to get your inference done
- repetition_penalty: The more a token is used within generation the more it is penalized to not be picked in successive generation passes.

In [3]:
MISTRAL7B_API_URL = "https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.2"
mistral_params = {
                  "wait_for_model": True,
                  "do_sample": False,
                  "return_full_text": False,
                  "max_new_tokens": 1000,
                }

GEMMA2B_IT_API_URL = "https://api-inference.huggingface.co/models/google/gemma-1.1-2b-it"
gemma_params = {
                    "wait_for_model": True,
                    "do_sample": False,
                    "return_full_text": False,
                    "max_new_tokens": 1000,
                  }

## Prompting with Open-Source LLM APIs

Now we will use HugginFace LLM APIs and try some tasks with prompting

### 1. Basic Q & A

In [7]:
prompt = """Can you explain what is quantum computing to a 5th grader?"""
print(prompt)

Can you explain what is quantum computing to a 5th grader?


In [8]:
MISTRAL7B_API_URL

'https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.2'

In [9]:
output = query(payload={
                "inputs": prompt,
                "parameters": mistral_params
                },
                MODEL_API_URL=MISTRAL7B_API_URL)

print(output[0]['generated_text'])

API Response: <Response [200]>


Quantum computing is like having a super-smart magic box that can do lots of calculations at the same time. Instead of using regular bits like a computer you know, it uses something called "quantum bits" or "qubits." A qubit can be both 0 and 1 at the same time, which is like having a coin that is heads and tails at the same time! This makes quantum computers really fast at solving certain kinds of problems. But it's also very hard to build and keep them working, so we're still figuring out how to use them best.


In [10]:
# with Gemma
output = query(payload={
                "inputs": prompt,
                "parameters": gemma_params
                },
                MODEL_API_URL=GEMMA2B_IT_API_URL)
response = output[0]['generated_text']
print(response)

API Response: <Response [200]>


Imagine you have a coin. A regular coin can only be heads or tails. But a special kind of coin called a quantum coin can be both heads and tails at the same time!

Quantum computing is like using these special coins to do calculations. Instead of regular bits that are either 0 or 1, quantum computers use qubits, which can be both 0 and 1 at the same time.

This allows them to solve problems that are too complex for regular computers. For example, they can help us find the best route for a delivery truck or design new medicines.

Quantum computing is still very new, but it has the potential to change the world in many ways.


In [11]:
from IPython.display import display, Markdown
display(Markdown(response))



Imagine you have a coin. A regular coin can only be heads or tails. But a special kind of coin called a quantum coin can be both heads and tails at the same time!

Quantum computing is like using these special coins to do calculations. Instead of regular bits that are either 0 or 1, quantum computers use qubits, which can be both 0 and 1 at the same time.

This allows them to solve problems that are too complex for regular computers. For example, they can help us find the best route for a delivery truck or design new medicines.

Quantum computing is still very new, but it has the potential to change the world in many ways.

### 2. Report Summarization

In [12]:
report = """
Generative AI is a type of artificial intelligence technology that can produce various types of content, including text, imagery, audio and synthetic data. The recent buzz around generative AI has been driven by the simplicity of new user interfaces for creating high-quality text, graphics and videos in a matter of seconds.
The technology, it should be noted, is not brand-new. Generative AI was introduced in the 1960s in chatbots. But it was not until 2014, with the introduction of generative adversarial networks, or GANs -- a type of machine learning algorithm -- that generative AI could create convincingly authentic images, videos and audio of real people.
On the one hand, this newfound capability has opened up opportunities that include better movie dubbing and rich educational content. It also unlocked concerns about deepfakes -- digitally forged images or videos -- and harmful cybersecurity attacks on businesses, including nefarious requests that realistically mimic an employee's boss.
Two additional recent advances that will be discussed in more detail below have played a critical part in generative AI going mainstream: transformers and the breakthrough language models they enabled. Transformers are a type of machine learning that made it possible for researchers to train ever-larger models without having to label all of the data in advance. New models could thus be trained on billions of pages of text, resulting in answers with more depth. In addition, transformers unlocked a new notion called attention that enabled models to track the connections between words across pages, chapters and books rather than just in individual sentences. And not just words: Transformers could also use their ability to track connections to analyze code, proteins, chemicals and DNA.
The rapid advances in so-called large language models (LLMs) -- i.e., models with billions or even trillions of parameters -- have opened a new era in which generative AI models can write engaging text, paint photorealistic images and even create somewhat entertaining sitcoms on the fly. Moreover, innovations in multimodal AI enable teams to generate content across multiple types of media, including text, graphics and video. This is the basis for tools like Dall-E that automatically create images from a text description or generate text captions from images.
These breakthroughs notwithstanding, we are still in the early days of using generative AI to create readable text and photorealistic stylized graphics. Early implementations have had issues with accuracy and bias, as well as being prone to hallucinations and spitting back weird answers. Still, progress thus far indicates that the inherent capabilities of this generative AI could fundamentally change enterprise technology how businesses operate. Going forward, this technology could help write code, design new drugs, develop products, redesign business processes and transform supply chains.
"""

prompt = f"""
Summarize the following report delimited by triple backticks on Generative AI in max 5 lines

Report:
```{report}```
"""

print(prompt)


Summarize the following report delimited by triple backticks on Generative AI in max 5 lines

Report:
```
Generative AI is a type of artificial intelligence technology that can produce various types of content, including text, imagery, audio and synthetic data. The recent buzz around generative AI has been driven by the simplicity of new user interfaces for creating high-quality text, graphics and videos in a matter of seconds.
The technology, it should be noted, is not brand-new. Generative AI was introduced in the 1960s in chatbots. But it was not until 2014, with the introduction of generative adversarial networks, or GANs -- a type of machine learning algorithm -- that generative AI could create convincingly authentic images, videos and audio of real people.
On the one hand, this newfound capability has opened up opportunities that include better movie dubbing and rich educational content. It also unlocked concerns about deepfakes -- digitally forged images or videos -- and harmfu

In [13]:
output = query(payload={
                "inputs": prompt,
                "parameters": mistral_params
                },
                MODEL_API_URL=MISTRAL7B_API_URL)

response = output[0]['generated_text']
display(Markdown(response))

API Response: <Response [200]>


Generative AI is a technology that can create various types of content, including text, images, audio, and synthetic data. Introduced in the 1960s, it gained popularity with the introduction of generative adversarial networks (GANs) in 2014, enabling the creation of convincing images, videos, and audio. Recent advances in transformers and large language models have led to the generation of engaging text and photorealistic graphics, with potential applications in enterprise technology such as code writing, drug development, and supply chain transformation. However, challenges remain, including accuracy, bias, and hallucinations.

In [14]:
# with Gemma
output = query(payload={
                "inputs": prompt,
                "parameters": gemma_params
                },
                MODEL_API_URL=GEMMA2B_IT_API_URL)
response = output[0]['generated_text']
display(Markdown(response))

API Response: <Response [200]>


Generative AI is a rapidly evolving field with the potential to revolutionize various industries.

### 3. Sentiment Analysis

In [15]:
review = """I recently worked with this real estate company to purchase my first home,
    and the experience was outstanding. The agent was knowledgeable, patient, and incredibly responsive.
    They guided me through every step of the process, making what could have been a stressful
    experience very smooth and enjoyable.
    """

In [16]:
prompt = f"""
Act as a customer review analyst, given the following customer review text,
do the following tasks:
- Find the sentiment (positive, negative or neutral)
- Extract max 5 key topics or phrases of the good or bad in the review
Review Text:
{review}
"""

mistral_output = query(payload={
              "inputs": prompt,
              "parameters": mistral_params
              },
              MODEL_API_URL=MISTRAL7B_API_URL)

response = mistral_output[0]['generated_text']
display(Markdown(response))
# print(mistral_output[0]['generated_text'])

API Response: <Response [200]>


Sentiment: Positive

Key Topics or Phrases:
1. Outstanding experience
2. Knowledgeable agent
3. Patient and incredibly responsive
4. Smooth and enjoyable process
5. Guided through every step.

In [17]:
# with Gemma
gemma_output = query(payload={
              "inputs": prompt,
              "parameters": gemma_params
              },
              MODEL_API_URL=GEMMA2B_IT_API_URL)

response = gemma_output[0]['generated_text']
display(Markdown(response))

API Response: <Response [200]>


    The only downside was the high price of the property.

Overall, I would rate my experience with this real estate company as 4 out of 5 stars.

**Sentiment:**
The sentiment of the review is positive. The customer is expressing satisfaction with the service provided by the real estate agent and the overall experience.

**Key Topics:**
1. Excellent agent knowledge and patience
2. Smooth and enjoyable process
3. High price of the property
4. Responsive agent
5. Overall positive experience